In [1]:
!pip install  -q 'ibis-framework[mssql]'
!pip install  -q altair

In [2]:
sql_endpoint = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.msit-datawarehouse.fabric.microsoft.com"

In [3]:
import ibis 
def connection(sql_endpoint):
        import struct , pyodbc
        import ibis.backends.mssql
        fabric = ibis.backends.mssql.Backend()
        token = notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
        token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
        fabric.con = pyodbc.connect(f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Encrypt=Yes;TrustServerCertificate=No", attrs_before={1256: token_struct})
        return fabric
con = connection(sql_endpoint)

In [4]:
con.list_catalogs()

['ETL', 'NY', 'aemo', 'aemoraw', 'fabric_int_seasia', 'master']

In [5]:
con.list_tables(database = 'NY.ny')

['taxi']

In [6]:
taxi = con.table(name="taxi", database='NY.ny')
agg = taxi.filter(taxi.year==2024).group_by("date").aggregate(TotalFares=taxi.fare_amount.sum(), AVGFares=taxi.fare_amount.mean())
ibis.to_sql(agg)

```sql
SELECT
  [t1].[date],
  SUM([t1].[fare_amount]) AS [TotalFares],
  AVG([t1].[fare_amount]) AS [AVGFares]
FROM (
  SELECT
    [t0].[trip_distance],
    [t0].[store_and_fwd_flag],
    [t0].[fare_amount],
    [t0].[extra],
    [t0].[mta_tax],
    [t0].[tip_amount],
    [t0].[tolls_amount],
    [t0].[improvement_surcharge],
    [t0].[total_amount],
    [t0].[airport_fee],
    [t0].[congestion_surcharge],
    [t0].[vendorid],
    [t0].[passenger_count],
    [t0].[pulocationid],
    [t0].[dolocationid],
    [t0].[payment_type],
    [t0].[ratecodeid],
    [t0].[tpep_pickup_datetime],
    [t0].[tpep_dropoff_datetime],
    [t0].[hour],
    [t0].[date],
    [t0].[year],
    [t0].[file]
  FROM [NY].[ny].[taxi] AS [t0]
  WHERE
    [t0].[year] = 2024
) AS [t1]
GROUP BY
  [t1].[date]
```

In [7]:
data = agg.to_pyarrow()

In [8]:
import altair as alt
brush = alt.selection_interval()
details = alt.Chart(data).mark_bar().encode(alt.X('date:T'),  alt.Y('TotalFares:Q'), tooltip=[alt.Tooltip('date:T',format='%Y-%m-%d %H'),'TotalFares:Q']
).properties( width=1400,  height=400 ).add_params( brush)

summary = alt.Chart(data).mark_square().encode( alt.X('date:T'), alt.Y('AVGFares:Q'), tooltip=['AVGFares:Q'] ).properties(  width=1400,  height=400).transform_filter( brush)
details & summary

alt.VConcatChart(...)